In [1]:

#pip install commands in case you are setting up a new environment

#!pip install haystack-ai
#!pip install "sentence-transformers>=2.2.0" "huggingface_hub>=0.22.0"
#!pip install markdown-it-py mdit_plain pypdf
#!pip install accelerate
#!pip install huggingface_hub


In [2]:
#Set the working directory and fetch the data (create the cmd to do it)

data_dir = "/home/vittorio/work/scripts/RAG/data/"

wget_knowledgebase_cmd = 'wget -P {} https://github.com/Mestizia/nfdi4plants.knowledgebase/archive/refs/heads/main.zip'.format(data_dir)
unzip_knowledgebase_cmd = 'unzip {}main.zip -d {}'.format(data_dir, data_dir)
mv_knowledgebase_cmd = 'mv {}nfdi4plants.knowledgebase-main {}'.format(data_dir, data_dir)

wget_ARC_specs_cmd = 'wget -P {} https://github.com/Mestizia/ARC-specification/archive/refs/heads/main.zip'.format(data_dir)
unzip_ARC_specs_cmd = 'unzip {}main.zip -d {}'.format(data_dir, data_dir)
mv_ARC_specs_cmd = 'mv {}ARC-specification-main {}'.format(data_dir, data_dir)

In [3]:
#download the files from https://github.com/Mestizia/nfdi4plants.knowledgebase/archive/refs/heads/main.zip and https://github.com/Mestizia/ARC-specification/archive/refs/heads/main.zip in the data_dir folder unzip it and move it

#!$wget_knowledgebase_cmd
#!$unzip_knowledgebase_cmd
#!$mv_knowledgebase_cmd

#!$wget_ARC_specs_cmd
#!$unzip_ARC_specs_cmd
#!$mv_ARC_specs_cmd

#!wget -P /home/vittorio/work/scripts/RAG/data/ https://github.com/Mestizia/nfdi4plants.knowledgebase/archive/refs/heads/main.zip

#!unzip /home/vittorio/work/scripts/RAG/data/main.zip -d /home/vittorio/work/scripts/RAG/data/

#!mv /home/vittorio/work/scripts/RAG/data/nfdi4plants.knowledgebase-main/* /home/vittorio/work/scripts/RAG/data/

#now add the files from https://github.com/Mestizia/ARC-specification/archive/refs/heads/main.zip

#!wget -P /home/vittorio/work/scripts/RAG/data/ https://github.com/Mestizia/ARC-specification/archive/refs/heads/main.zip

#!unzip /home/vittorio/work/scripts/RAG/data/main.zip.1 -d /home/vittorio/work/scripts/RAG/data/

#!mv /home/vittorio/work/scripts/RAG/data/ARC-specification-main/* /home/vittorio/work/scripts/RAG/data/



In [4]:
# Lots of imports, I am sure we can reduce them in the future once we settle on things

import os

from haystack.components.writers import DocumentWriter
from haystack.components.converters import MarkdownToDocument, TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack import components
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import MarkdownToDocument
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceAPIGenerator


In [5]:
#log in huggingface, save the access token in the data folder as hf_access_token.txt

if os.path.isfile('{}hf_access_token.txt'.format(data_dir)):
    with open('{}hf_access_token.txt'.format(data_dir), 'r') as file:
        hf_access_token = file.read().replace('\n', '')

from huggingface_hub.hf_api import HfFolder
HfFolder.save_token(hf_access_token)



In [6]:
#get all the markdown and txt files in the data folder

document_paths = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".md") or file.endswith(".txt"):
            document_paths.append(os.path.join(root, file))

print (document_paths)

['/home/vittorio/work/scripts/RAG/data/ARC specification.md', '/home/vittorio/work/scripts/RAG/data/README.md', '/home/vittorio/work/scripts/RAG/data/CONTRIBUTING.md', '/home/vittorio/work/scripts/RAG/data/ISA-XLSX.md', '/home/vittorio/work/scripts/RAG/data/hf_access_token.txt', '/home/vittorio/work/scripts/RAG/data/src/docs/Home.md', '/home/vittorio/work/scripts/RAG/data/src/docs/image-index-crawler.md', '/home/vittorio/work/scripts/RAG/data/src/docs/image-index.md', '/home/vittorio/work/scripts/RAG/data/src/docs/CONTRIBUTING.md', '/home/vittorio/work/scripts/RAG/data/src/docs/fundamentals/ElectronicLabNotebooks.md', '/home/vittorio/work/scripts/RAG/data/src/docs/fundamentals/ResearchDataManagement.md', '/home/vittorio/work/scripts/RAG/data/src/docs/fundamentals/PersistentIdentifiers.md', '/home/vittorio/work/scripts/RAG/data/src/docs/fundamentals/DataPublications.md', '/home/vittorio/work/scripts/RAG/data/src/docs/fundamentals/Ontologies.md', '/home/vittorio/work/scripts/RAG/data/src

In [7]:
#initialize the document store in memory and create the pipeline

document_store = InMemoryDocumentStore()

pipeline = Pipeline()
pipeline.add_component("converter", MarkdownToDocument())
pipeline.add_component("cleaner", DocumentCleaner())
pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=5))
pipeline.add_component("writer", DocumentWriter(document_store=document_store))
pipeline.connect("converter", "cleaner")
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter", "writer")

pipeline.run({"converter": {"sources": document_paths}})

Converting markdown files to Documents:   0%|          | 0/576 [00:00<?, ?it/s]

Converting markdown files to Documents: 100%|██████████| 576/576 [00:03<00:00, 146.18it/s]


{'writer': {'documents_written': 3599}}

In [8]:
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever

retriever = InMemoryBM25Retriever(document_store=document_store)

In [9]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

embedder = SentenceTransformersDocumentEmbedder("sentence-transformers/all-MiniLM-L6-v2")


In [10]:
from haystack.components.builders.answer_builder import AnswerBuilder

builder = AnswerBuilder()


In [11]:
# add reader

from haystack.components.readers import ExtractiveReader

reader = ExtractiveReader()
reader.warm_up()



In [12]:


querying_pipeline = Pipeline()
querying_pipeline.add_component(instance=retriever, name="Retriever")
querying_pipeline.add_component(instance=reader, name="Reader")
querying_pipeline.add_component(instance=AnswerBuilder(), name='Answer_builder')
querying_pipeline.connect("Retriever.documents", "Reader.documents")


🚅 Components
  - Retriever: InMemoryBM25Retriever
  - Reader: ExtractiveReader
  - Answer_builder: AnswerBuilder
🛤️ Connections
  - Retriever.documents -> Reader.documents (List[Document])

In [13]:

template = """
Answer the questions based on the given context.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ question }}
Answer:
"""
pipe = Pipeline()
pipe.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
pipe.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component(
    "llm",
    HuggingFaceAPIGenerator(api_type="serverless_inference_api", api_params={"model": "HuggingFaceH4/zephyr-7b-beta"}),
)

pipe.connect("embedder.embedding", "retriever.query_embedding")
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")



🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceAPIGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [14]:
question = (
    "what is the ISA format?"
)

pipe.run(
    {
        "embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_new_tokens": 50}},
    }
)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

No Documents found with embeddings. Returning empty list. To generate embeddings, use a DocumentEmbedder.


{'llm': {'replies': [' The ISA format is a standard format for storing and sharing scientific data. It allows researchers to easily share their data with others, making it more accessible and reusable. The format is widely used in the earth sciences, and is supported by'],
  'meta': [{'model': 'HuggingFaceH4/zephyr-7b-beta',
    'finish_reason': 'length',
    'usage': {'completion_tokens': 50}}]}}